In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os

import numpy as np
import pandas as pd
import atecml.data
from tqdm import tqdm

class BasicModel(object):
    """Parent class of basic models"""
    def train(self, x_train, y_train, x_val, y_val):
        """return a trained model and eval metric o validation data"""
        pass
    
    def predict(self, model, x_test):
        """return the predicted result"""
        pass
    
    def get_oof(self, x_train, y_train, x_test, n_folds = 5):
        """K-fold stacking"""
        num_train, num_test = x_train.shape[0], x_test.shape[0]
        oof_train = np.zeros((num_train,)) 
        oof_test = np.zeros((num_test,))
        oof_test_all_fold = np.zeros((num_test, n_folds))
        aucs = []
        model_list = []
        
        for i in range(0,n_folds):
            
            val_index = DateFold[5] #始终用最后20%验证            
            train_index = list(all_list - DateFold[i])
            
            y_tra1 = y_train[train_index]
            n_sample = y_tra1.shape[0]
            n_pos_sample = y_tra1[y_tra1 == 0].shape[0]
            n_neg_sample = y_tra1[y_tra1 == 1].shape[0]
            print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample,
                                                   n_pos_sample / n_sample,
                                                               n_neg_sample / n_sample))
            target = 'label'
            temp_df = train_df[train_df.index.isin(train_index)].reset_index(drop=True)
            
            filter_list = [x for x in temp_df.columns if x not in atecml.data.NOT_FEATURE_COLUMNS]
            filter_list.append(target)
            temp_df = temp_df[filter_list]
            
            fraud_indices = np.array(temp_df[temp_df[target] == 1].index)
            normal_indices = np.array(temp_df[temp_df[target] == 0].index)
            
            number_record_fraud = len(fraud_indices)
            number_record_normal = len(normal_indices)
            #undersample
            random_normal_indices = np.array(np.random.choice(normal_indices,number_record_fraud,replace=False))
            #oversample
            random_fraud_indices = np.array(np.random.choice(fraud_indices,number_record_normal,replace=True))
            # 汇总正、负样本的索引
            under_sample_indices = np.concatenate([fraud_indices,random_normal_indices])
            over_sample_indices = np.concatenate([normal_indices,random_fraud_indices])
            
            # 根据汇总的索引提取数据集
            #under_sample_data = temp_df.iloc[under_sample_indices,:]
            under_sample_data = temp_df.iloc[over_sample_indices,:]
            # 在数据集中提取特征、标签数据
            x_tra = np.array(under_sample_data.iloc[:,under_sample_data.columns != target])
            y_tra = np.array(under_sample_data.iloc[:,under_sample_data.columns == target][target])
            
            # 检查获取的样本特征、标签数据
            #print(x_tra.shape,y_tra.shape)
                            
            print('{0} fold, train {1}, val {2}'.format(i, len(train_index), len(val_index)))
            
            x_val, y_val = x_train[val_index], y_train[val_index]

            model, auc = self.train(x_tra, y_tra, x_val, y_val)
            aucs.append(auc)
            model_list.append(model)
            oof_train[val_index] = self.predict(model, x_val)
            oof_test_all_fold[:, i] = self.predict(model, x_test)
        oof_test = np.mean(oof_test_all_fold, axis=1)
        print('all aucs {0}, average {1}'.format(aucs, np.mean(aucs)))
        return oof_train, oof_test,model_list

import lightgbm as lgb
class LGBClassifier(BasicModel):
    '''
    ' 调参范围
    'num_leaves':range(35,65,5)
    'learning_rate':[0.01,0.05,0.1,0.3,0.5,0.7]
    'min_child_weight':range(1,6,2)
    'max_depth':range(3,10,2),
    'subsample':[i/10.0 for i in range(6,10)],正常直接设置为1
    'colsample_bytree':[i/10.0 for i in range(6,10)]，正常直接设置为1
    'reg_alpha','reg_lambda':[1e-5, 1e-2, 0.1, 1, 2,2.5,3]
    '''
    def __init__(self,boost_type,boost_round=1000,early_stop=100,pos_weight=1):
        self.num_boost_round = boost_round
        self.early_stopping_rounds = early_stop
        self.params = {
            'task': 'train',
            'boosting_type': boost_type,
            'colsample_bytree': 0.7,
            'learning_rate': 0.1,
            'max_bin': 255,
            'max_depth': -1,
            'metric': {'auc'},
            'min_child_samples': 600,
            'min_child_weight': 0.05,
            'min_split_gain': 0,
            'nthread': 40,
            'num_leaves': 80,
            'objective': 'binary',
            'reg_alpha': 0.1,
            'reg_lambda': 0.1,
            #'is_unbalance':'true',
            #'scale_pos_weight': pos_weight,
            'subsample': 0.85,
            'subsample_for_bin': 200000,
            'subsample_freq': 1,
            'use_missing': 'true',
            'verbose' : -1,
            }
        print(self.params)
        
    def train(self, x_train, y_train, x_val, y_val):
        print('train with lgb model')
        lgbtrain = lgb.Dataset(x_train, y_train)
        lgbval = lgb.Dataset(x_val, y_val)
        model = lgb.train(self.params, 
                          lgbtrain,
                          valid_sets=lgbval, 
                          verbose_eval = 50,
                          num_boost_round = self.num_boost_round,
                          early_stopping_rounds = self.early_stopping_rounds)
        return model, model.best_score['valid_0']['auc']
    
    def predict(self, model, x_test):
        print('test with lgb model')
        return model.predict(x_test, num_iteration=model.best_iteration)

def stack_layer1_result(X_train,rf_model_list,gbdt_model_list,dart_model_list):
    with atecml.data.timer('Classification: Building Layer-1 Stack'):
        rf_input_list = []
        for idx in tqdm(range(len(rf_model_list))):
            model = rf_model_list[idx]
            _temp_df = model.predict(X_train,num_iteration=model.best_iteration)
            rf_input_list.append(pd.DataFrame(_temp_df))
        rf_oof_predict= np.array(pd.concat(rf_input_list,ignore_index=True,axis=1).mean(axis=1))    
    
        gbdt_input_list = []
        for idx in tqdm(range(len(gbdt_model_list))):
            model = gbdt_model_list[idx]
            _temp_df = model.predict(X_train,num_iteration=model.best_iteration)
            gbdt_input_list.append(pd.DataFrame(_temp_df))
        gbdt_oof_predict= np.array(pd.concat(gbdt_input_list,ignore_index=True,axis=1).mean(axis=1))
        
        
        dart_input_list = []
        for idx in tqdm(range(len(dart_model_list))):
            model = dart_model_list[idx]
            _temp_df = model.predict(X_train,num_iteration=model.best_iteration)
            dart_input_list.append(pd.DataFrame(_temp_df))
        dart_oof_predict= np.array(pd.concat(dart_input_list,ignore_index=True,axis=1).mean(axis=1))
    
    input_predict = [rf_oof_predict, gbdt_oof_predict, dart_oof_predict] 
    stacked_predict = np.concatenate([f.reshape(-1, 1) for f in input_predict], axis=1)
    
    return stacked_predict  


In [2]:
#训练集为第一步build的纬度提升矩阵，并过滤掉unknown标签
data = atecml.data.load_train()
train_df = data[data['label']!=-1].reset_index(drop=True)

#最终预测的测试集为unknown标签
val_df = data[data['label']==-1].reset_index(drop=True)


predictors = [x for x in data.columns if x not in atecml.data.NOT_FEATURE_COLUMNS2]
DateFold={}

DateFold[0] = set(atecml.data.filter_date(train_df,start_date='2017-09-05',end_date='2017-09-13').index)
DateFold[1] = set(atecml.data.filter_date(train_df,start_date='2017-09-14',end_date='2017-09-22').index)
DateFold[2] = set(atecml.data.filter_date(train_df,start_date='2017-09-23',end_date='2017-10-01').index)
DateFold[3] = set(atecml.data.filter_date(train_df,start_date='2017-10-02',end_date='2017-10-12').index)
DateFold[4] = set(atecml.data.filter_date(train_df,start_date='2017-10-13',end_date='2017-10-22').index)
DateFold[5] = list(atecml.data.filter_date(train_df,start_date='2017-10-23',end_date='2017-11-24').index)

all_list = set(train_df.index) - set(DateFold[5])
len(all_list),len(DateFold[5])

(761864, 228142)

In [3]:
target='label'
x_train = np.array(train_df[predictors])
y_train = np.array(train_df[target])
x_test = np.array(val_df[predictors])
print(x_train.shape, y_train.shape, x_test.shape)

(990006, 230) (990006,) (4725, 230)


In [4]:
num_pos = np.sum(train_df[target])  
num_neg = x_train.shape[0]- num_pos
scale_pos_weight =  num_neg/num_pos
print(num_pos,num_neg,scale_pos_weight)

12122 977884 80.67018643788154


In [5]:
num_boost_round = 1000
num_early_stop = 100

In [ ]:

# get output of first layer models and construct as input for the second layer          
rf_classifier = LGBClassifier(boost_type='rf',boost_round=num_boost_round,early_stop=num_early_stop,pos_weight= scale_pos_weight)
rf_oof_train, rf_oof_test,rf_model_list = rf_classifier.get_oof(x_train, y_train, x_test)
print(rf_oof_train.shape, rf_oof_test.shape)  


{'min_child_samples': 600, 'min_child_weight': 0.05, 'objective': 'binary', 'use_missing': 'true', 'verbose': -1, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'max_bin': 255, 'learning_rate': 0.1, 'metric': {'auc'}, 'subsample_for_bin': 200000, 'min_split_gain': 0, 'nthread': 40, 'max_depth': -1, 'num_leaves': 80, 'subsample_freq': 1, 'boosting_type': 'rf', 'subsample': 0.85, 'colsample_bytree': 0.7, 'task': 'train'}
样本个数：613958; 正样本占98.77%; 负样本占1.23%
0 fold, train 613958, val 228142
train with lgb model
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.679846
[100]	valid_0's auc: 0.690284
[150]	valid_0's auc: 0.695156
[200]	valid_0's auc: 0.691843
[250]	valid_0's auc: 0.694868
Early stopping, best iteration is:
[151]	valid_0's auc: 0.695424
test with lgb model
test with lgb model
样本个数：621410; 正样本占98.77%; 负样本占1.23%
1 fold, train 621410, val 228142
train with lgb model
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.588344


In [ ]:
gbdt_classifier = LGBClassifier(boost_type='gbdt',boost_round=num_boost_round,early_stop=num_early_stop,pos_weight= scale_pos_weight)
gbdt_oof_train, gbdt_oof_test,gbdt_model_list = gbdt_classifier.get_oof(x_train, y_train, x_test)
print(gbdt_oof_train.shape, gbdt_oof_test.shape)  

dart_classifier = LGBClassifier(boost_type='dart',boost_round=num_boost_round,early_stop=num_early_stop,pos_weight= scale_pos_weight)
dart_oof_train, dart_oof_test,dart_model_list = dart_classifier.get_oof(x_train, y_train, x_test)
print(dart_oof_train.shape, dart_oof_test.shape)    

In [ ]:
stacked_train = stack_layer1_result(x_train,rf_model_list,gbdt_model_list,dart_model_list)
stacked_test = stack_layer1_result(x_test,rf_model_list,gbdt_model_list,dart_model_list)

In [ ]:
# use XGBOOST  as the model of the second layer
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

model = XGBClassifier(
 learning_rate =0.05,
 n_estimators=200,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.9,
 objective= 'binary:logistic',
 scoring='roc_auc',
 scale_pos_weight= scale_pos_weight,
 nthread=40,
 seed=27)


# split for validation
n = int(stacked_train.shape[0] * 0.8)
x_tra, y_tra = stacked_train[:n], y_train[:n]
x_val, y_val = stacked_train[n:], y_train[n:]
model.fit(x_tra,y_tra)
y_pred = pd.DataFrame(model.predict_proba(x_val))[1]

_f1,_f2,_f3 = atecml.data.accuracy_validation(y_val,y_pred)

In [ ]:
# predict on test data
final_model = XGBClassifier(
 learning_rate =0.05,
 n_estimators=200,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.9,
 objective= 'binary:logistic',
 scoring='roc_auc',
 scale_pos_weight= scale_pos_weight,
 nthread=40,
 seed=27)


final_model.fit(stacked_train, y_train)
test_prediction = final_model.predict_proba(stacked_test)

result=pd.DataFrame()
result['id'] = val_df['id']
result['score'] = pd.DataFrame(test_prediction)[1]
result.to_pickle('./reject_inf.dat')

In [ ]:
result.hist(bins=100)

In [ ]:
len(result[result.score > 0.8])

In [ ]:
result[result.score > 0.8].hist()

In [ ]:
result.to_pickle('./reject_inf.dat')

In [ ]:
aval_df = atecml.data.load_test()

In [ ]:
ax_test = np.array(aval_df[predictors])

In [ ]:
len(ax_test)

In [ ]:
stacked_test = stack_layer1_result(ax_test,rf_model_list,gbdt_model_list,dart_model_list)

In [ ]:
len(stacked_test)

In [ ]:
test_prediction = final_model.predict_proba(stacked_test)

result=pd.DataFrame()
result['id'] = aval_df['id']
result['score'] = pd.DataFrame(test_prediction)[1]

In [ ]:
result.to_csv('./aaa.csv',index=False)